<a href="https://colab.research.google.com/github/tomonari-masada/course2023-stats1/blob/main/03_text_retrieval_with_multinomial_distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多項分布を使った文書検索

## 説明
* 検索対象の各文書について、最尤推定で単語確率を求める。
  * $p_{d,w}$: 文書$d$における単語$w$の出現確率
* クエリの尤度を、各文書について求めた単語確率を使って計算する。
  * $n_{q,w}$: クエリ$q$における単語$w$の出現頻度
  * このとき、文書$d$の単語確率を使ったクエリ$q$の対数尤度は、以下の通り。
$$\begin{align}
L_q(d) = \sum_w n_{q,w} \log p_{d,w}
\end{align}$$
* このように計算されたクエリの尤度によって、検索対象の文書をソートする。
  * $L_q(d)$が大きい順に、文書を検索結果として表示する。
* 上記の方法では検索があまりうまくいかないことを確認する。
  * 上の式を使うと、検索対象の文書に出現しない単語を含むクエリの尤度はゼロ（対数尤度はマイナス無限大）になる。

## 準備

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multinomial
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

## データセット

* 20 newsgroupsコーパスを使う。
* テストセットの文書をクエリだと思って、訓練セットの文書を検索する。

### データセットのダウンロード

In [ ]:
train_corpus, train_labels = fetch_20newsgroups(subset="train", return_X_y=True)
test_corpus, test_labels = fetch_20newsgroups(subset="test", return_X_y=True)

In [ ]:
print(train_corpus[0])

In [ ]:
len(train_corpus), len(test_corpus)

### 各文書での単語の出現回数を数える

* ここでは、訓練データで5回未満しか出現しない単語と、英語のストップワードとを、無視する。

In [ ]:
vectorizer = CountVectorizer(min_df=5, stop_words="english")
X_train = vectorizer.fit_transform(train_corpus).toarray()
X_test = vectorizer.transform(test_corpus).toarray()

In [ ]:
X_train.shape, X_test.shape

### 語彙集合を取得する

In [ ]:
vocabulary = vectorizer.get_feature_names_out()
print(vocabulary)

## 最尤推定

* 各文書を多項分布でモデリングする。
* そして、最尤推定により、単語確率パラメータの値を推定する。

In [ ]:
X_train_probs = X_train / X_train.sum(axis=1).reshape(-1, 1)

In [ ]:
X_train_probs.sum(axis=1)

## 対数尤度を求めるヘルパ関数

* テスト文書の対数尤度を、特定の訓練文書の単語確率を使って求めるヘルパ関数
* 訓練文書にない単語を含むテスト文書は、対数尤度がマイナス無限大になる

In [ ]:
def log_likelihood(x_test, x_train_prob):
  # クエリが訓練文書にない単語を含む場合、マイナス無限大を返す
  if (x_test * (x_train_prob == 0)).sum() > 0:
    return - np.inf
  # 多項分布の作成
  rv = multinomial(x_test.sum(), x_train_prob)
  # 対数尤度を確率質量関数を使って計算
  return rv.logpmf(x_test)

## 検索の実行

* クエリとして使うテスト文書の設定

In [ ]:
query_idx = 100

 * 個々の訓練文書ごとに、クエリ文書の対数尤度を計算

In [ ]:
score = list()
for i in range(X_train.shape[0]):
  score.append(log_likelihood(X_test[query_idx], X_train_probs[i]))
score = np.array(score)

# ゼロスコアはマイナス無限大で置き換える
score = np.where(score == 0.0, - np.inf, score)

# 降順にソート
sorted_train_indices = (- score).argsort()

* クエリ文書の内容を確認

In [ ]:
print(test_corpus[query_idx])

* 検索結果1位の文書の内容を確認

In [ ]:
print(train_corpus[sorted_train_indices[0]])

* それぞれのカテゴリを確認
  * 同じカテゴリである方が、もちろん、望ましい。

In [ ]:
test_labels[query_idx], train_labels[sorted_train_indices[0]]

* クエリと1位の文書で共有されている単語を確認

In [ ]:
vocabulary[(X_test[query_idx] * (X_train_probs[sorted_train_indices[0]] > 0)) > 0]

## MAP推定


* 各訓練文書について、MAP推定で単語確率を推定する。

In [ ]:
# ディリクレ事前分布のパラメータ
beta = 0.01

X_train = X_train + beta
X_train_probs = X_train / X_train.sum(axis=1).reshape(-1, 1)

## 検索の実行

In [ ]:
# 個々の訓練文書ごとに、クエリ文書のスコアを計算
score = list()
for i in range(X_train.shape[0]):
  score.append(log_likelihood(X_test[query_idx], X_train_probs[i]))
score = np.array(score)

# 降順にソート
sorted_train_indices = (- score).argsort()

In [ ]:
print(test_corpus[query_idx])

In [ ]:
print(train_corpus[sorted_train_indices[0]])

In [ ]:
test_labels[query_idx], train_labels[sorted_train_indices[0]]

In [ ]:
vocabulary[(X_test[query_idx] * (X_train_probs[sorted_train_indices[0]] > 0)) > 0]